In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/imdb-dataset-of-50k-movie-reviews


# Load dataset

In [3]:
import pandas as pd

# Load dataset (adjust the path based on dataset choice)
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

# Check the first few rows
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Preprocess the Data

In [4]:
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Text cleaning function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove punctuation and numbers
    return text

df['cleaned_text'] = df['review'].apply(clean_text)

# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['cleaned_text'])
sequences = tokenizer.texts_to_sequences(df['cleaned_text'])

# Padding
max_length = 200
X = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

# Encode labels (positive=1, negative=0)
y = np.array([1 if label == 'positive' else 0 for label in df['sentiment']])


# Create and Train LSTM Model

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Define the model
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=max_length),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X, y, epochs=5, batch_size=32, validation_split=0.2)


Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.5315 - loss: 0.6854 - val_accuracy: 0.6490 - val_loss: 0.6286
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.6275 - loss: 0.6400 - val_accuracy: 0.5571 - val_loss: 0.6763
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.7164 - loss: 0.5552 - val_accuracy: 0.8363 - val_loss: 0.3892
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.8568 - loss: 0.3537 - val_accuracy: 0.8535 - val_loss: 0.3358
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - accuracy: 0.8924 - loss: 0.2746 - val_accuracy: 0.8670 - val_loss: 0.3173


In [8]:
model.save('sentiment_lstm.h5')